# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 73 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.22416


extracting tarball to tmp_2410.22416...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.22537


extracting tarball to tmp_2410.22537...

...

Retrieving document from  https://arxiv.org/e-print/2410.23171


extracting tarball to tmp_2410.23171...

 done.


X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Found 53 bibliographic references in tmp_2410.23171/main.bbl.
Issues with the citations
syntax error in line 172: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.23171-b31b1b.svg)](https://arxiv.org/abs/2410.23171) | **Dust extinction-curve variation in the translucent interstellar medium is driven by PAH growth**  |
|| <mark>X. Zhang</mark>, B. S. Hensley, G. M. Green |
|*Appeared on*| *2024-10-31*|
|*Comments*| *11 pages, 6 figures, submitted on October 30, 2024*|
|**Abstract**|            The first all-sky, high-resolution, 3D map of the optical extinction curve of the Milky Way (Zhang & Green 2024) revealed an unexpected steepening of the extinction curve in the moderate-density, "translucent" interstellar medium (ISM). We argue that this trend is driven by growth of polycyclic aromatic hydrocarbons (PAHs) through gas-phase accretion. We find a strong anti-correlation between the slope of the optical extinction curve -- parameterized by $R(V)$ -- and maps of PAH abundance -- parameterized by $q_{\rm PAH}$ -- derived from infrared emission. The range of observed $q_{\rm PAH}$ indicates PAH growth by a factor of $\sim$2 between $A_V \simeq 1$ and 3. This implies a factor-of-two stronger 2175 Angstrom feature, which is sufficient to lower $R(V)$ by the observed amount. This level of PAH growth is possible given rapid accretion timescales and the depletion of carbon in the translucent ISM. Spectral observations by JWST would provide a definitive test of this proposed explanation of $R(V)$ variation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.22416-b31b1b.svg)](https://arxiv.org/abs/2410.22416) | **ERGO-ML: A continuous organization of the X-ray galaxy cluster population in TNG-Cluster with contrastive learning**  |
|| U. Chadayammuri, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-10-31*|
|*Comments*| *Please see more results from TNG-Cluster on astro-ph this week from Rohr+ and Prunier+*|
|**Abstract**|            The physical properties of the intracluster medium (ICM) reflect signatures of the underlying gravitational potential, mergers and strong interactions with other halos and satellite galaxies, as well as galactic feedback from supernovae and supermassive black holes (SMBHs). Traditionally, clusters have been characterized in terms of summary statistics, such as halo mass, X-ray luminosity, cool-core state, luminosity of AGN, and number of merging components. In this paper of the Extracting Reality from Galaxy Observables with Machine Learning series (ERGO-ML), we instead consider the full information content available in maps of X-ray emission from the ICM. We employ Nearest Neighbour Contrastive Learning (NNCLR) to identify and populate a low-dimensional representation space of such images. Using idealized X-ray maps of the 352 clusters of the TNG-Cluster cosmological magnetohydrodynamical simulation suite, we take three orthogonal projections of each cluster at eight snapshots within the redshift range $0\leq z<1$, resulting in a dataset of $\sim$8,000 images. Our findings reveal that this representation space forms a continuous distribution from relaxed to merging objects, and from centrally-peaked to flat emission profiles. The representation also exhibits clear trends with redshift, with halo, gas, stellar, and SMBH mass, with time since a last major merger, and with indicators of dynamical state. We show that an 8-dimensional representation can be used to predict a variety of cluster properties, find analogs, and identify correlations between physical properties, thereby suggesting causal relationships. Our analysis demonstrates that contrastive learning is a powerful tool for characterizing galaxy clusters from their images alone, allowing us to derive constraints on their physical properties and formation histories using cosmological hydrodynamical galaxy simulations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.22537-b31b1b.svg)](https://arxiv.org/abs/2410.22537) | **The Dust Extinction Curve: Beyond R(V)**  |
|| G. M. Green, <mark>X. Zhang</mark>, <mark>R. Zhang</mark> |
|*Appeared on*| *2024-10-31*|
|*Comments*| *19 pages, 13 figures; data available at DOI:https://doi.org/10.5281/zenodo.14005028*|
|**Abstract**|            The dust extinction curve is typically parameterized by a single variable, R(V), in optical and near-infrared wavelengths. R(V) controls the slope of the extinction-vs.-wavelength curve, and is thought to reflect the grain-size distribution and composition of dust. Low-resolution, flux-calibrated BP/RP spectra from Gaia have allowed the determination of the extinction curve along sightlines to 130 million stars in the Milky Way and Magellanic Clouds. We show that these extinction curves contain more than a single degree of freedom - that is, that they are not simply described by R(V). We identify a number of components that are orthogonal to R(V) variation, and show that these components vary across the sky in coherent patterns that resemble interstellar medium structure. These components encode variation in the 770 nm extinction feature, intermediate-scale and very broad structure, and a newly identified feature at 850 nm, and likely trace both dust composition and local conditions in the interstellar medium. Correlations of the 770 nm and 850 nm features with R(V) suggest that their carriers become more abundant as the carrier of the 2175 Angstrom feature is destroyed. Our 24 million extinction-curve decompositions and feature equivalent-width measurements are publicly available at this https URL.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \C at position 12</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.23171/./MC_Rv_qPAH.png', 'tmp_2410.23171/./Aquila_S_no_bottom_label.png', 'tmp_2410.23171/./California_no_label.png', 'tmp_2410.23171/./Ursa_Ma_no_top_label.png', 'tmp_2410.23171/./Rv_qPAH_cor.png']
copying  tmp_2410.23171/./MC_Rv_qPAH.png to _build/html/
copying  tmp_2410.23171/./Aquila_S_no_bottom_label.png to _build/html/
copying  tmp_2410.23171/./California_no_label.png to _build/html/
copying  tmp_2410.23171/./Ursa_Ma_no_top_label.png to _build/html/
copying  tmp_2410.23171/./Rv_qPAH_cor.png to _build/html/
exported in  _build/html/2410.23171.md
    + _build/html/tmp_2410.23171/./MC_Rv_qPAH.png
    + _build/html/tmp_2410.23171/./Aquila_S_no_bottom_label.png
    + _build/html/tmp_2410.23171/./California_no_label.png
    + _build/html/tmp_2410.23171/./Ursa_Ma_no_top_label.png
    + _build/html/tmp_2410.23171/./Rv_qPAH_cor.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Dust extinction-curve variation in the translucent interstellar medium is driven by PAH growth

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.23171-b31b1b.svg)](https://arxiv.org/abs/2410.23171)<mark>Appeared on: 2024-10-31</mark> -  _11 pages, 6 figures, submitted on October 30, 2024_

</div>
<div id="authors">

<mark>X. Zhang</mark>, B. S. Hensley, G. M. Green

</div>
<div id="abstract">

**Abstract:** The first all-sky, high-resolution, 3D map of the optical extinction curve of the Milky Way \citep{ZG24a} revealed an unexpected steepening of the extinction curve in the moderate-density, "translucent" interstellar medium (ISM). We argue that this trend is driven by growth of polycyclic aromatic hydrocarbons (PAHs) through gas-phase accretion. We find a strong anti-correlation between the slope of the optical extinction curve -- parameterized by $R(V)$ -- and maps of PAH abundance -- parameterized by $q_{\rm PAH}$ -- derived from infrared emission. The range of observed $q_{\rm PAH}$ indicates PAH growth by a factor of $\sim$ 2 between $A_V \simeq 1$ and 3. This implies a factor-of-two stronger 2175 Å feature, which is sufficient to lower $R(V)$ by the observed amount. This level of PAH growth is possible given rapid accretion timescales and the depletion of carbon in the translucent ISM. Spectral observations by JWST would provide a definitive test of this proposed explanation of $R(V)$ variation.

</div>

<div id="div_fig1">

<img src="tmp_2410.23171/./MC_Rv_qPAH.png" alt="Fig1" width="100%"/>

**Figure 1. -** Quantitative comparison on the per-star basis between $R(V)$ and $q_\mathrm{PAH}$ in LMC (left panel) and SMC (right panel). The median value of $R(V)$ decreases by $0.4$ as $q_\mathrm{PAH}$ increases from $1\%$ to $6\%$ in the LMC. In the SMC, the trend is not clear because of small variation range of $q_\mathrm{PAH}$ as well as the difference in spatial coverage of both maps.
     (*fig:quant_Rv_qPAH*)

</div>
<div id="div_fig2">

<img src="tmp_2410.23171/./Aquila_S_no_bottom_label.png" alt="Fig3.1" width="33%"/><img src="tmp_2410.23171/./California_no_label.png" alt="Fig3.2" width="33%"/><img src="tmp_2410.23171/./Ursa_Ma_no_top_label.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Comparison of 2D distribution of $A(V)$, $R(V)$, $q_\mathrm{PAH}$ for dust clouds. For $A(V)$ and $R(V)$ maps, we select high-quality stars from \cite{ZG24a}, and remove lines of sight where only partial extinction is covered by XP. For $q_\mathrm{PAH}$ map, we remove lines of sight where the $q_\mathrm{PAH}$ map is of low quality ($\chi^2/\mathrm{DOF}>3$ or $q_\mathrm{PAH}/\sigma(q_\mathrm{PAH})<3$), or that cross multiple clouds (Equation \ref{eqn:multi_cloud}). The binning method and selection of stars in the 2D maps are described in Appendix \ref{app:binning}. Despite different baselines and finer structures in different maps, all clouds tend to have lower $R(V)$ at high-$q_\mathrm{PAH}$ regions, as indicated quantitatively on the per-star basis in Figure \ref{fig:quant_Rv_qPAH} (*fig:Av_Rv_qPAH*)

</div>
<div id="div_fig3">

<img src="tmp_2410.23171/./Rv_qPAH_cor.png" alt="Fig5" width="100%"/>

**Figure 5. -** Correlation between $R(V)$ and $q_\mathrm{PAH}$ using stars that pass all the cuts in Figure \ref{fig:Av_Rv_qPAH}. 16th, 50th and 84th percentiles are in cyan lines. We also label $R(V)=3.1$, the Milky Way average value, in red dotted lines.  Although different clouds have different baselines of $R(V)$, due to different initial distribution of dust grain size, all dust clouds show anti-correlation between $R(V)$ and $q_\mathrm{PAH}$.  (*fig:quant_Rv_qPAH*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.23171"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

236  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
